# Crypto bot metrics evaluation:

## Test data extraction from Binance

In [4]:
from src.get_historical_data import get_historical_data

df = get_historical_data(
    coin='BTCUSDT',
    lookback=365*6
)
df

,date,BTCUSDT
0,2019-03-09,3943.04
1,2019-03-10,3916.82
2,2019-03-11,3871.61
3,2019-03-12,3882.73
4,2019-03-13,3866.00
...,...,...
2185,2025-03-02,94270.00
2186,2025-03-03,86220.61
2187,2025-03-04,87281.98
2188,2025-03-05,90606.01


### Plotting data results:

In [5]:
import plotly.express as px
px.line(df, x='date', y='BTCUSDT')

## Extracting exchange interest from YFinance

In [6]:
from src.get_treasury_rate import get_treasury_rate

df2 = get_treasury_rate(lookback=365*6)
df2

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


,date,treasury_rate
0,2019-03-08,2.393
1,2019-03-11,2.380
2,2019-03-12,2.395
3,2019-03-13,2.388
4,2019-03-14,2.388
...,...,...
1502,2025-02-27,4.195
1503,2025-02-28,4.193
1504,2025-03-03,4.193
1505,2025-03-04,4.195


## Testing metrics

In [7]:
from src.crypto_metrics import CryptoMetrics
metrics = CryptoMetrics(lookback=365*6)
rsi = metrics.calculate_rsi(df=df, coin='BTCUSDT')

### RSI:

In [8]:
fig = px.line(rsi, x='date', y='BTCUSDT', title='BTCUSDT RSI', labels={'BTCUSDT': 'BTCUSDT Price', 'date': 'Date'})

# Add green dots for rsi_buy_BTCUSDT
fig.add_scatter(x=rsi[rsi['rsi_buy_BTCUSDT'] == 1]['date'], 
                y=rsi[rsi['rsi_buy_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='RSI Buy')

# Add red dots for rsi_sell_BTCUSDT
fig.add_scatter(x=rsi[rsi['rsi_sell_BTCUSDT'] == 1]['date'], 
                y=rsi[rsi['rsi_sell_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='RSI Sell')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Treasury Rate correlation:

In [9]:
from importlib import reload
import src.crypto_metrics
reload(src.crypto_metrics)
from src.crypto_metrics import CryptoMetrics
treasury_corr = metrics.calculate_corr_treasury(df=df, coin='BTCUSDT')
treasury_corr

[*********************100%***********************]  1 of 1 completed


,date,BTCUSDT,treasury_rate,treasury_corr_BTCUSDT,treasury_corr_buy_BTCUSDT,treasury_corr_sell_BTCUSDT
0,2019-03-09,3943.04,NaN,NaN,False,False
1,2019-03-10,3916.82,NaN,NaN,False,False
2,2019-03-11,3871.61,2.380,NaN,False,False
3,2019-03-12,3882.73,2.395,NaN,False,False
4,2019-03-13,3866.00,2.388,NaN,False,False
...,...,...,...,...,...,...
2185,2025-03-02,94270.00,4.193,0.081990,False,False
2186,2025-03-03,86220.61,4.193,0.127204,False,False
2187,2025-03-04,87281.98,4.195,0.142969,False,False
2188,2025-03-05,90606.01,4.205,0.168629,False,False


In [10]:
px.line(treasury_corr, x='date', y='treasury_corr_BTCUSDT')

In [11]:
fig = px.line(
    treasury_corr, 
    x='date', 
    y='BTCUSDT', 
    title='BTCUSDT correlation with interest rate USA', 
    labels= {
        'BTCUSDT': 'BTCUSDT Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_BTCUSDT
fig.add_scatter(x=treasury_corr[treasury_corr['treasury_corr_buy_BTCUSDT'] == 1]['date'], 
                y=treasury_corr[treasury_corr['treasury_corr_buy_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='Treasury rate correlation buy signal')

# Add red dots for rsi_sell_BTCUSDT
fig.add_scatter(x=treasury_corr[treasury_corr['treasury_corr_sell_BTCUSDT'] == 1]['date'], 
                y=treasury_corr[treasury_corr['treasury_corr_sell_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='Treasury rate correlation sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Bollinger Bands:

In [12]:
bb = metrics.calculate_bollinger_bands(df=df, coin='BTCUSDT')
bb

,date,BTCUSDT,std_BTCUSDT,mean_BTCUSDT,top_band_BTCUSDT,bottom_band_BTCUSDT,bb_buy_BTCUSDT,bb_sell_BTCUSDT
0,2019-03-09,3943.04,NaN,3943.040000,NaN,NaN,0,0
1,2019-03-10,3916.82,18.540340,3929.930000,3957.740510,3902.119490,0,0
2,2019-03-11,3871.61,36.133266,3910.490000,3964.689899,3856.290101,0,0
3,2019-03-12,3882.73,32.604647,3903.550000,3952.456970,3854.643030,0,0
4,2019-03-13,3866.00,32.852667,3896.040000,3945.319001,3846.760999,0,0
...,...,...,...,...,...,...,...,...
2185,2025-03-02,94270.00,5235.870087,92359.109333,100212.914463,84505.304203,0,0
2186,2025-03-03,86220.61,5350.707544,91699.275333,99725.336650,83673.214017,0,0
2187,2025-03-04,87281.98,5337.647679,91132.740667,99139.212186,83126.269148,0,0
2188,2025-03-05,90606.01,5188.110818,90795.025333,98577.191560,83012.859106,0,0


In [13]:
fig = px.line(
    bb, 
    x='date', 
    y='BTCUSDT', 
    title='Bollinger Bands for BTCUSDT', 
    labels= {
        'BTCUSDT': 'BTCUSDT Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_BTCUSDT
fig.add_scatter(x=bb[bb['bb_buy_BTCUSDT'] == 1]['date'], 
                y=bb[bb['bb_buy_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='Bollinger Bands buy signal')

# Add red dots for rsi_sell_BTCUSDT
fig.add_scatter(x=bb[bb['bb_sell_BTCUSDT'] == 1]['date'], 
                y=bb[bb['bb_sell_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='Bollinger Bands sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Mean Average Convergence Divergence (MACD):

In [14]:
macd = metrics.calculate_macd(df=df, coin='BTCUSDT')
macd

,date,BTCUSDT,EMA12_BTCUSDT,EMA26_BTCUSDT,MACD_BTCUSDT,Signal_Line_BTCUSDT,MACD_buy_BTCUSDT,MACD_sell_BTCUSDT
0,2019-03-09,3943.04,3943.040000,3943.040000,0.000000,0.000000,0,0
1,2019-03-10,3916.82,3939.006154,3941.097778,-2.091624,-0.418325,0,1
2,2019-03-11,3871.61,3928.637515,3935.950535,-7.313020,-1.797264,0,0
3,2019-03-12,3882.73,3921.574820,3932.008273,-10.433453,-3.524502,0,0
4,2019-03-13,3866.00,3913.024848,3927.118771,-14.093924,-5.638386,0,0
...,...,...,...,...,...,...,...,...
2185,2025-03-02,94270.00,90689.145825,93510.013693,-2820.867868,-2370.375422,0,0
2186,2025-03-03,86220.61,90001.678775,92970.057864,-2968.379089,-2489.976155,0,0
2187,2025-03-04,87281.98,89583.263579,92548.718763,-2965.455184,-2585.071961,0,0
2188,2025-03-05,90606.01,89740.609182,92404.814410,-2664.205228,-2600.898614,0,0


In [15]:
fig = px.line(
    macd, 
    x='date', 
    y='BTCUSDT', 
    title='MACD for BTCUSDT', 
    labels= {
        'BTCUSDT': 'BTCUSDT Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_BTCUSDT
fig.add_scatter(x=macd[macd['MACD_buy_BTCUSDT'] == 1]['date'], 
                y=macd[macd['MACD_buy_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='MACD buy signal')

# Add red dots for rsi_sell_BTCUSDT
fig.add_scatter(x=macd[macd['MACD_sell_BTCUSDT'] == 1]['date'], 
                y=macd[macd['MACD_sell_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='MACD sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Final buy setup:

In [16]:
df_buy = metrics.set_buy(df, coin='BTCUSDT')
df_buy

[*********************100%***********************]  1 of 1 completed


,date,BTCUSDT,buy_BTCUSDT,sell_BTCUSDT
0,2019-03-09,3943.04,0,0
1,2019-03-10,3916.82,0,0
2,2019-03-11,3871.61,0,0
3,2019-03-12,3882.73,0,0
4,2019-03-13,3866.00,0,0
...,...,...,...,...
2185,2025-03-02,94270.00,0,0
2186,2025-03-03,86220.61,0,0
2187,2025-03-04,87281.98,0,0
2188,2025-03-05,90606.01,0,0


In [17]:
fig = px.line(
    df_buy, 
    x='date', 
    y='BTCUSDT', 
    title='Buy setup for BTCUSDT', 
    labels= {
        'BTCUSDT': 'BTCUSDT Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_BTCUSDT
fig.add_scatter(x=df_buy[df_buy['buy_BTCUSDT'] == 1]['date'], 
                y=df_buy[df_buy['buy_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='Buy signal')

# Add red dots for rsi_sell_BTCUSDT
fig.add_scatter(x=df_buy[df_buy['sell_BTCUSDT'] == 1]['date'], 
                y=df_buy[df_buy['sell_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='Sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

## Evaluate model return:

To evaluate the model return, it is possible to use a simulation of a bot working on the operations of trading for a given historic period of the coin price. Based on that, every trade will be set to operate with 10 dolars. By the end of the period, it will be computed the return of that.

Every buy will be 10 dollars
Every sell will be 10 dollars

The start value will be 1000 dollars

In [18]:
import pandas as pd

def simulate_model(df: pd.DataFrame, initial_capital: float, trade_value: float, coin: str):
    balance = initial_capital  # Initial money in USD
    coin_holdings = 0  # Number of coins owned

    for _, row in df.iterrows():
        price = row[coin]  # Get current coin price

        if balance < 0:
            print("Bankrupt!")
            break
        
        # Sell logic: Sell all holdings when a sell signal appears
        if row[f'sell_{coin}'] == 1 and coin_holdings > 0:
            coins_sold = trade_value / price  # Sell $10 worth of the coin
            if coins_sold > coin_holdings:  
                coins_sold = coin_holdings  # Prevent selling more than we own
            coin_holdings -= coins_sold
            balance += coins_sold * price


        # Buy logic: Buy $10 worth of the coin when a buy signal appears
        if row[f'buy_{coin}'] == 1 and balance >= trade_value:
            coins_bought = trade_value / price  # Calculate how many coins we can buy
            coin_holdings += coins_bought
            balance -= trade_value  # Reduce balance

    # Final value: Cash + value of remaining coins
    final_balance = balance + (coin_holdings * df[coin].iloc[-1])
    
    print(f"Initial Capital: ${initial_capital:.2f}")
    print(f"Final Balance: ${final_balance:.2f}")
    print(f"Total Profit/Loss: ${final_balance - initial_capital:.2f}")
    
    return final_balance

# Example usage:
# df should contain a column with the coin's price and buy/sell signals.
simulate_model(df_buy, initial_capital=1000, trade_value=10, coin="BTCUSDT")

Initial Capital: $1000.00
Final Balance: $1216.96
Total Profit/Loss: $216.96


np.float64(1216.955705008357)

#### Calculating ROI:

$$ ROI = (\frac{\text{Final Balance} - \text{Initial Capital}}{\text{Initial Capital}})\times 100$$

In [19]:
ROI = ((simulate_model(df_buy, initial_capital=1000, trade_value=10, coin="BTCUSDT") - 1000) / 1000)*100
print(f"ROI: {ROI:.2f}%")

Initial Capital: $1000.00
Final Balance: $1216.96
Total Profit/Loss: $216.96
ROI: 21.70%
